In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def cal_upsampling(depth, rows, cols, strides, kernel_size, padding, output_padding):
    
    print(f'strides -- {strides}')
    print(f'kernel -- {kernel_size}')
    print(f'padding -- {padding}')
    new_depth = ((depth - 1) * strides + kernel_size[0] - 2 * padding +
    output_padding)
    new_rows = ((rows - 1) * strides + kernel_size[1] - 2 * padding +
    output_padding)
    new_cols = ((cols - 1) * strides + kernel_size[1] - 2 * padding +
    output_padding)
    
    return new_depth, new_rows, new_cols

In [5]:
def doubleConvBlock(x, numFilters):

    # Conv3D then ReLU activation
    x = layers.Conv3D(numFilters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    # Conv3D then ReLU activation
    x = layers.Conv3D(numFilters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)

    return x

In [6]:
def downsampleBlock(x, numFilters):
    f = doubleConvBlock(x, numFilters)
    p = layers.MaxPool3D(2)(f)
    p = layers.Dropout(0.3)(p)

    return f, p

In [7]:
def upsampleBlock(x, convFeatures, numFilters, kernel_size):
    # upsample
    x = layers.Conv3DTranspose(numFilters, kernel_size, 2)(x)
    # concatenate 
    x = layers.concatenate([x, convFeatures])
    # dropout
    x = layers.Dropout(0.1)(x)
    # Conv2D twice with ReLU activation
    x = doubleConvBlock(x, numFilters)

    return x

In [2]:
def Unet(width=10, height=10, depth=13):

    # inputs
    inputs = keras.Input((width, height, depth, 1))

    # encoder: contracting path - downsample
    # 1 - downsample
    f1, p1 = downsampleBlock(inputs, 32)
    #print(f'Shape of f1 -- {f1.shape} & Shape of p1 -- {p1.shape}')
    
    # 2 - downsample
    f2, p2 = downsampleBlock(p1, 64)
    #print(f'Shape of f2 -- {f2.shape} & Shape of p2 -- {p2.shape}')
    
    # 3 - bottleneck
    bottleneck = doubleConvBlock(p2, 128)
    #print(f'Shape of bottleneck {bottleneck.shape}')
    
    # decoder: expanding path - upsample
    # 4 - upsample
    u1 = upsampleBlock(bottleneck, f2, 64, (3,3,2))
    
    # 5 - upsample
    #print(f'Shape of u6 -- {u6.shape} & Shape of p2 -- {f1.shape}')
    u2 = upsampleBlock(u1, f1, 32, (2,2,3))
    
    # 6 - 1x1 convolutions
    o1 = layers.Conv3D(16, 1, padding='same', activation='relu')(u2)
    o2 = layers.Conv3D(4, 1, padding='same', activation='relu')(o1)
    
    # 7 - layer for voxel segmentation
    segmentOutputs = layers.Conv3D(1, 1, padding='same', activation='sigmoid')(o2)
    
    # dense network for classfication
    f3 = layers.Dense(units=64, activation="relu")(f2)
    f4 = layers.Dropout(0.1)(f3)
    f5 = layers.Dense(units=32, activation="relu")(f4)
    classOutputs = layers.Dense(units=1, activation="sigmoid")(f5)

    #outputs = layers.Dense(units=1, activation="sigmoid")(f4)

    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, [classOutputs, segmentOutputs], name="U-Net")
    #class_model = tf.keras.Model(inputs, classOutputs, name="U-Net-class")

    return unet_model

In [15]:
model = Unet(width=10, height=10, depth=13)
model.summary()

Shape of f1 -- (None, 10, 10, 13, 32) & Shape of p1 -- (None, 5, 5, 6, 32)
Shape of f2 -- (None, 5, 5, 6, 64) & Shape of p2 -- (None, 2, 2, 3, 64)
Shape of bottleneck (None, 2, 2, 3, 128)
Shape of u6 -- (None, 5, 5, 6, 64) & Shape of p2 -- (None, 10, 10, 13, 32)
Model: "U-Net-segment"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 10, 10, 13,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d_42 (Conv3D)             (None, 10, 10, 13,   896         ['input_4[0][0]']                
                                32)                                                               
                     

total = 1300
weight_0 = (1/1299) * (total/n)
weight_1 = (1/1) * (total/n)
class_weight = {0: weight_for_0, 1: weight_for_1}

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_features, val_labels),
    class_weight=class_weight)

loss function only over non zero voxels?